In [1]:
import duckdb
from dotenv import load_dotenv
import os
import time
from deltalake import write_deltalake

load_dotenv()

True

In [2]:
account_conn_string = os.getenv("ACCOUNT_CONN_STRING")

storage_options = {
    "account_name": os.getenv("ACCOUNT_NAME"),
    "account_key": os.getenv("ACCOUNT_KEY")
}

In [3]:
con = duckdb.connect()
con.execute(f""" 
    CREATE SECRET secret1 (
    TYPE azure,
    CONNECTION_STRING '{account_conn_string}'
);
""")

#con.execute("INSTALL delta; INSTALL azure; LOAD azure; LOAD delta;")

### Carga de las canciones

Obtenemos tres dataframes:

1. Todas las canciones en el top 50 diario para todos los paises
2. El total de apariciones de cada canción en el top 50 diario para España.
3. Todos los top 1 de caciones en el periodo de tiempo para España.

In [4]:
path = "abfs://spotify-songs/*.csv"

start = time.perf_counter()

songs_df = (
    con.sql(f"""
            SELECT 
                album_name,
                name,
                artists,
                album_release_date,
                daily_rank,
                daily_movement,
                weekly_movement,
                CASE WHEN country = '' THEN 'WO' ELSE country END AS country, 
                snapshot_date,
                popularity,
                CAST(duration_ms / 1000 AS INT) AS duration_seconds
            FROM read_csv('{path}');
    """)
)

total_aparences_top_50_es_df = (
    con.sql("""
            SELECT
                album_name,
                name,
                artists,
                count(1) as total_aparences
            FROM songs_df
            WHERE country = 'ES'
            GROUP BY 1, 2, 3
            ORDER BY total_aparences DESC
    """)
)

top_first_per_day_es_df = (
    con.sql("""
            SELECT
                album_name,
                name,
                artists,
                snapshot_date
            FROM songs_df
            WHERE country = 'ES' AND daily_rank = 1
            ORDER BY snapshot_date
    """)
)

### Guardamos en el lago de datos

In [5]:
path_songs = "abfs://spotify-delta-lake/duckdb/top_fifty_songs_daily"
write_deltalake(path_songs, songs_df.arrow(), mode="overwrite", storage_options=storage_options)

path_total_aparences = "abfs://spotify-delta-lake/duckdb/total_aparences_es"
write_deltalake(path_total_aparences, total_aparences_top_50_es_df.arrow(), mode="overwrite", storage_options=storage_options)

path_top_first_per_day = "abfs://spotify-delta-lake/duckdb/top_first_per_day_es"
write_deltalake(path_top_first_per_day, top_first_per_day_es_df.arrow(), mode="overwrite", storage_options=storage_options)



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [6]:
end = time.perf_counter()

print(f"Tiempo de ejecución: {end - start:.6f} segundos")

Tiempo de ejecución: 76.173705 segundos
